In [1]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import requests
from bs4 import BeautifulSoup
import time 
from tqdm import tqdm
import random
import pandas as pd

In [2]:
def get_sitemap_urls(sitemap_url):
    Pro=[]
    for i in sitemap_url:
        response = requests.get(i)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            urls = [loc.text for loc in soup.find_all('loc')]
            Pro.extend(urls) 
        else:
            print(f"Failed to fetch sitemap. Status Code: {response.status_code}")
    return Pro     

In [3]:
def unique(Orignal_list):
    unique_list = [item for index, item in enumerate(Orignal_list) if item not in Orignal_list[:index]]
    return unique_list

In [4]:
def scrollWindow():
    time.sleep(2)
    initial_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        driver.implicitly_wait(2) 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == initial_height:
            break
        initial_height = new_height
       

In [39]:
def get_product_info(URL):
    driver.get(URL)
    scrollWindow()
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    try:
        Product_element = soup.find('h1', class_="product__title")
        Product = Product_element.text
    except:
        Product_element = soup.find('h5', class_="product__title")
        Product = Product_element.text

    Rating_element = soup.find('span',class_="avg-score font-color-gray-darker")
    Rating = Rating_element.text if Rating_element else '0'
    ReviewCount_element = soup.find('span', class_="reviews-qa-label font-color-gray")
    ReviewCount = ReviewCount_element.text.split(',')[0] if ReviewCount_element else '0 Reviews'
    df=[]
    skipped = []
    form_class =soup.find_all('div' , class_="selector-wrapper") 
    def creating_row(Form=None, Style= None, Size= None):
        try:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            SKU_element=  soup.find('span', class_='current-sku')
            SKU = SKU_element.text
        except:
            SKU_element = driver.find_element(By.XPATH, './/div[1]/div/div/div[3]/div[1]/div[1]/div/div/span/span')
            SKU = SKU_element.text
        try:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            Price_element=  soup.find('span', class_='h4')
            Price = Price_element.text
        except:
            Price_element = driver.find_element(By.XPATH, './/div[1]/div/div/div[3]/div[1]/div[2]/div/span/span[1]')
            Price = Price_element.text
        Variant = []
        if Form is not None:
            Variant.append(Form)
        if Style is not None:
            Variant.append(Style)
        if Size is not None:
            Variant.append(Size)
        row={
             'PageURL': URL, 
            'Product': Product,
            'SKU': SKU,
            'Variant': '|'.join(Variant),
            'Price': Price,
            'ReviewCount': ReviewCount,
            'Rating': Rating
        }
        return row
    if len(form_class)>=1:
        for i in range(len(soup.find_all('div' , class_="selector-wrapper")[0].find_all('input'))):
            Form_element = driver.find_element(By.XPATH, './/div[2]/div[1]/fieldset/span[{}]/label/span'.format(i+1))
            driver.execute_script("arguments[0].click();", Form_element)
            Form = Form_element.text
            if len(form_class)>=2:
                for j in range(len(soup.find_all('div' , class_="selector-wrapper")[1].find_all('input'))):
                    Size_element = driver.find_element(By.XPATH, './/div[2]/div[2]/fieldset/span[{}]/label/span'.format(j+1))
                    driver.execute_script("arguments[0].click();", Size_element)
                    Size = Size_element.text
                    if len(form_class)==3:
                        for k in reversed(range(len(soup.find_all('div' , class_="selector-wrapper")[2].find_all('input')))):
                            Style_element = driver.find_element(By.XPATH, './/div[2]/div[3]/fieldset/span[{}]/label/span'.format(k+1))
                            driver.execute_script("arguments[0].click();", Style_element)
                            Style = Style_element.text
                            row = creating_row(Form=Form, Style= Style, Size=Size)
                            df.append(row)
                    else:
                        row = creating_row(Form=Form, Style= None, Size= Size)
                        df.append(row)
                        
            else:
                row = creating_row(Form=Form, Style= None, Size= None)
                df.append(row)
    else:
        row = creating_row(Form=None, Style= None, Size= None)
        df.append(row)
    return df

In [6]:
sitemap_url = ['https://www.harrisseeds.com/sitemap_products_1.xml?from=8434003665&to=6592712900680',
              'https://www.harrisseeds.com/sitemap_products_2.xml?from=6592753631304&to=6792758427720']
sitemap_urls = get_sitemap_urls(sitemap_url)
Pro_URL= []
for i in sitemap_urls:
    if 'https://www.harrisseeds.com/products' in i:
        Pro_URL.append(i)
df = pd.DataFrame(Pro_URL)
df.to_csv('Links.csv',index=False)

C:\Users\Rahul Manocha\anaconda3\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [7]:
allowed_user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/14.1.2",
    "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"   
]

In [40]:
Product_details= []
#Pro_URL.index('')

'https://www.harrisseeds.com/products/06445-onion-plant-collection-1-1800'

In [41]:
len(unique(Pro_URL))

2908

In [50]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[2668:], desc="GettingData", unit="URL"):
    df = get_product_info(i)
    Product_details.extend(df)
    #print(df)
daf = pd.DataFrame(Product_details)
daf.to_csv('HarrisSeed.csv',index=False)
driver.quit()

GettingData: 100%|██████████████████████████████████████████████████████████████████| 240/240 [27:33<00:00,  6.89s/URL]


In [44]:
daf = pd.DataFrame(Product_details)
daf.to_csv('HarrisSeeds.csv',index=False)

In [46]:
driver.quit()

In [48]:
len(Product_details)

7649